In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA

In [32]:
def run_all_models(X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded):
    models = {
        "LogisticRegression": LogisticRegression(),
        "SVC": SVC(),
        "DecisionTreeClassifier": DecisionTreeClassifier(random_state=42),
        "RandomForestClassifier": RandomForestClassifier(random_state=42),
        "KNeighborsClassifier": KNeighborsClassifier(),
        "GradientBoostingClassifier": GradientBoostingClassifier(random_state=42),
        "GaussianNB": GaussianNB(),
    }

    # Create an empty DataFrame to store accuracy
    model_performance = []

    for name, model in models.items():
        model.fit(X_train_scaled, y_train_encoded)
        predictions = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test_encoded, predictions)
        print(f"{name} Accuracy:", accuracy)
        model_performance.append({"Model": name, "Accuracy": accuracy})

    # For the Sequential model
    sequential_model = Sequential()
    sequential_model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
    sequential_model.add(Dense(32, activation='relu'))
    sequential_model.add(Dense(1, activation='sigmoid'))
    sequential_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    sequential_model.fit(X_train_scaled, y_train_encoded, epochs=50, batch_size=10, verbose=1)
    loss, accuracy = sequential_model.evaluate(X_test_scaled, y_test_encoded)
    print(f"Sequential Model Accuracy: {accuracy}")
    print(f"Sequential Model Loss: {loss}")
    model_performance.append({"Model": "Sequential", "Accuracy": accuracy})

    return pd.DataFrame(model_performance)

In [70]:
df = pd.read_csv("Breast_Cancer.csv")
df

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,Regional,9,Positive,Positive,1,1,49,Alive
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,Regional,46,Positive,Positive,14,8,69,Alive
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Negative,11,3,69,Alive
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,44,Positive,Positive,11,1,72,Alive


In [34]:
df.isna().sum()

Age                       0
Race                      0
Marital Status            0
T Stage                   0
N Stage                   0
6th Stage                 0
differentiate             0
Grade                     0
A Stage                   0
Tumor Size                0
Estrogen Status           0
Progesterone Status       0
Regional Node Examined    0
Reginol Node Positive     0
Survival Months           0
Status                    0
dtype: int64

In [35]:
df.duplicated().sum()

1

In [71]:
df.drop_duplicates(inplace=True)

In [37]:
df

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,Regional,9,Positive,Positive,1,1,49,Alive
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,Regional,46,Positive,Positive,14,8,69,Alive
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Negative,11,3,69,Alive
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,44,Positive,Positive,11,1,72,Alive


In [38]:
df["Race"].unique()

array(['White', 'Black', 'Other'], dtype=object)

In [39]:
df["Status"].unique()

array(['Alive', 'Dead'], dtype=object)

In [40]:
df["Estrogen Status"].unique()

array(['Positive', 'Negative'], dtype=object)

In [41]:
df["Progesterone Status"].unique()

array(['Positive', 'Negative'], dtype=object)

In [42]:
df["differentiate"].unique()

array(['Poorly differentiated', 'Moderately differentiated',
       'Well differentiated', 'Undifferentiated'], dtype=object)

In [43]:
df["A Stage"].unique()

array(['Regional', 'Distant'], dtype=object)

In [44]:
df["Marital Status"].unique()

array(['Married', 'Divorced', 'Single ', 'Widowed', 'Separated'],
      dtype=object)

In [72]:
df["Status"] = df["Status"].apply(lambda x: 1 if x =="Alive" else 0)
df["Estrogen Status"] = df["Estrogen Status"].apply(lambda x: 1 if x =="Positive" else 0)
df["Progesterone Status"] = df["Progesterone Status"].apply(lambda x: 1 if x =="Positive" else 0)
df["A Stage"] = df["A Stage"].apply(lambda x: 1 if x =="Regional" else 0)

In [46]:
df

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,1,4,1,1,24,1,60,1
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,1,35,1,1,14,5,62,1
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,1,63,1,1,14,7,75,1
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,1,18,1,1,2,1,84,1
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,1,41,1,1,3,1,50,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,1,9,1,1,1,1,49,1
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,1,46,1,1,14,8,69,1
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,1,22,1,0,11,3,69,1
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,1,44,1,1,11,1,72,1


In [73]:
features_to_encode = ["Race", "Marital Status", "T Stage ","N Stage", "6th Stage","differentiate", "Grade"]

In [74]:
# Instantiate the encoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(df[features_to_encode])

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(features_to_encode))

# Reset index of your original DataFrame if it's not in default sequential order
df.reset_index(drop=True, inplace=True)

# Concatenate the original DataFrame with the new encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Optionally, drop the original columns
df.drop(features_to_encode, axis=1, inplace=True)

/Users/test/miniforge3/envs/machine-learning-env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [49]:
X, y = df.drop("Status", axis=1), df["Status"]


X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

models = run_all_models(X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded)
models

LogisticRegression Accuracy: 0.8956521739130435
SVC Accuracy: 0.8869565217391304
DecisionTreeClassifier Accuracy: 0.8496894409937888
RandomForestClassifier Accuracy: 0.9043478260869565
KNeighborsClassifier Accuracy: 0.8608695652173913
GradientBoostingClassifier Accuracy: 0.906832298136646
GaussianNB Accuracy: 0.8149068322981367
Epoch 1/50
322/322 [==============================] - 0s 569us/step - loss: 0.3476 - accuracy: 0.8633
Epoch 2/50
322/322 [==============================] - 0s 650us/step - loss: 0.2823 - accuracy: 0.8925
Epoch 3/50
322/322 [==============================] - 0s 577us/step - loss: 0.2704 - accuracy: 0.8996
Epoch 4/50
322/322 [==============================] - 0s 496us/step - loss: 0.2640 - accuracy: 0.9062
Epoch 5/50
322/322 [==============================] - 0s 511us/step - loss: 0.2576 - accuracy: 0.9065
Epoch 6/50
322/322 [==============================] - 0s 575us/step - loss: 0.2526 - accuracy: 0.9058
Epoch 7/50
322/322 [==============================] - 0s 5

,Model,Accuracy
0,LogisticRegression,0.895652
1,SVC,0.886957
2,DecisionTreeClassifier,0.849689
3,RandomForestClassifier,0.904348
4,KNeighborsClassifier,0.860870
5,GradientBoostingClassifier,0.906832
6,GaussianNB,0.814907
7,Sequential,0.869565


In [50]:
X, y = df.drop("Status", axis=1), df["Status"]


X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()

X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

models = run_all_models(X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded)
models

LogisticRegression Accuracy: 0.8956521739130435
SVC Accuracy: 0.8832298136645963
DecisionTreeClassifier Accuracy: 0.8496894409937888
RandomForestClassifier Accuracy: 0.9043478260869565
KNeighborsClassifier Accuracy: 0.8583850931677018
GradientBoostingClassifier Accuracy: 0.906832298136646
GaussianNB Accuracy: 0.8149068322981367
Epoch 1/50
322/322 [==============================] - 0s 587us/step - loss: 0.3851 - accuracy: 0.8462
Epoch 2/50
322/322 [==============================] - 0s 659us/step - loss: 0.3172 - accuracy: 0.8813
Epoch 3/50
322/322 [==============================] - 0s 531us/step - loss: 0.2962 - accuracy: 0.8937
Epoch 4/50
322/322 [==============================] - 0s 502us/step - loss: 0.2859 - accuracy: 0.8922
Epoch 5/50
322/322 [==============================] - 0s 501us/step - loss: 0.2823 - accuracy: 0.8996
Epoch 6/50
322/322 [==============================] - 0s 506us/step - loss: 0.2769 - accuracy: 0.8975
Epoch 7/50
322/322 [==============================] - 1s 2

,Model,Accuracy
0,LogisticRegression,0.895652
1,SVC,0.883230
2,DecisionTreeClassifier,0.849689
3,RandomForestClassifier,0.904348
4,KNeighborsClassifier,0.858385
5,GradientBoostingClassifier,0.906832
6,GaussianNB,0.814907
7,Sequential,0.881988


In [55]:
X, y = df.drop("Status", axis=1), df["Status"]


X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)


In [56]:
lst = []

for component in reversed(range(1, len(df.columns[:-1]))):
  pca = PCA(n_components=component)
  X_train_scaled = pca.fit_transform(X_train_scaled)
  X_test_scaled = pca.transform(X_test_scaled)
  models = run_all_models(X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded)
  lst.append(models)


LogisticRegression Accuracy: 0.8956521739130435
SVC Accuracy: 0.8869565217391304
DecisionTreeClassifier Accuracy: 0.8360248447204969
RandomForestClassifier Accuracy: 0.8869565217391304
KNeighborsClassifier Accuracy: 0.8608695652173913
GradientBoostingClassifier Accuracy: 0.8881987577639752
GaussianNB Accuracy: 0.8298136645962733
Epoch 1/50
322/322 [==============================] - 1s 769us/step - loss: 0.3502 - accuracy: 0.8645
Epoch 2/50
322/322 [==============================] - 0s 524us/step - loss: 0.2852 - accuracy: 0.8928
Epoch 3/50
322/322 [==============================] - 0s 490us/step - loss: 0.2705 - accuracy: 0.8962
Epoch 4/50
322/322 [==============================] - 0s 580us/step - loss: 0.2631 - accuracy: 0.9006
Epoch 5/50
322/322 [==============================] - 0s 704us/step - loss: 0.2549 - accuracy: 0.9027
Epoch 6/50
322/322 [==============================] - 0s 608us/step - loss: 0.2519 - accuracy: 0.9099
Epoch 7/50
322/322 [==============================] - 0s 

ValueError: Found array with 0 feature(s) (shape=(3218, 0)) while a minimum of 1 is required by LogisticRegression.